In [8]:
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression
import sklearn.feature_extraction.text as sktext
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [9]:
restored = pd.read_json('cleaned_data.json.gz')
restored.head(5)

,class,review_markup
0,neutral,"Холодильником користуюсь близько трьох тижнів,..."
1,positive,Любителям класики пораджу придбання двокамерно...
2,positive,Місяць вже як користуюся холодильником Купувал...
3,positive,Все чудово працює..стильний і гарна ціна!
4,positive,Питання до користувачів цим холодильником або ...


In [7]:
X_train, X_test, y_train, y_test = train_test_split(restored['review_markup'], restored['class'], test_size=0.25, random_state=42, stratify=restored['class'])

In [15]:
X_train.name = 'train'
X_test.name = 'test'

In [11]:
def preprocess(text):
    return text
dictionary_counter = sktext.CountVectorizer(preprocessor=preprocess, analyzer='char', ngram_range=(1, 4))
dictionary_counter.fit_transform(X_train)
def to_features(text):
    return dictionary_counter.transform(text)

In [12]:
def create_predictor(X_train, y_train):
    predictor = LogisticRegression()
    predictor.fit(to_features(X_train), y_train)
    return predictor

In [13]:
def predict(predictor, X):
    return predictor.predict(to_features(X))

In [17]:
y_train

382     positive
492     positive
63      positive
757     negative
991     positive
1156    positive
739     positive
1125    positive
619     positive
891     positive
356     positive
1012    positive
940     positive
1053    positive
72      positive
930     positive
1296    negative
89      positive
1129    positive
827     positive
650      neutral
1263    positive
290     positive
1229    positive
1305    positive
1085    positive
1101    positive
1037    positive
175     positive
1312    positive
          ...   
96      positive
1088    positive
137     positive
551     positive
500     positive
380     positive
1213    positive
328     positive
145     positive
10      positive
1093    positive
227     positive
1002    negative
423     positive
126     positive
723     positive
737     positive
887     negative
367      neutral
94      positive
243     positive
1104    positive
86      positive
123     positive
270      neutral
211     positive
705     positive
569     negati

In [25]:
def evaluate(label, predictor, X, y):
    print("{}. Evaluating on {}.".format(label, X.name))
    y_pred = predict(predictor, X)
    accuracy = metrics.accuracy_score(y, y_pred)
    print(metrics.confusion_matrix(y, y_pred, labels=['positive', 'negative', 'neutral']))
    print("Accuracy: {}".format(accuracy))
    return accuracy

In [26]:
predictor = create_predictor(X_train, y_train)
evaluate("Vanilla Logistic Regression, no preprocessing, default char ngrams 1-4", predictor, X_train, y_train)
evaluate("Vanilla Logistic Regression, no preprocessing, default char ngrams 1-4", predictor, X_test, y_test)

Vanilla Logistic Regression, no preprocessing, default char ngrams 1-4. Evaluating on train.
[[873   0   0]
 [  0  87   0]
 [  0   0  43]]
Accuracy: 1.0
Vanilla Logistic Regression, no preprocessing, default char ngrams 1-4. Evaluating on test.
[[289   2   0]
 [ 15  14   0]
 [  4   3   8]]
Accuracy: 0.9283582089552239


0.9283582089552239

TODO:
    - кросвалідація замість тестової виборки
    - використайте тональний словник для покращення якості класифікатора; поміряйте якість на тестовій вибірці
    - спробуйте покращити якість роботи класифікатора іншими способами (фільтрування стоп-слів, використання лем слів, ваші варіанти)
    - висновки